In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import fake_useragent
import re
import unicodedata
import warnings
warnings.filterwarnings("ignore")

ua = fake_useragent.UserAgent()

In [2]:
def get_url(tot_pages):
    for page in range(1,tot_pages+1):
        if page % log_frequency == 0:
            print(f'Log: working over page {page}...')       

        url_main = f"https://msk.fitnesslook.ru/trenajery/grebnye/?page={page}"
        response = requests.get(url_main, headers = {'user-agent': ua.random}, verify=False)
        soup = BeautifulSoup(response.content, 'lxml')
        data = soup.find_all(class_ = 'p_n notbm')
        for item in data:
            url = 'https://msk.fitnesslook.ru' + item['href']
            yield url

In [3]:
def get_item(category, tot_pages):
    count = 0
    for url_item in get_url(tot_pages):
        count += 1
        response = requests.get(url_item, headers = {'user-agent': ua.random}, verify=False)
        soup = BeautifulSoup(response.content, 'lxml')

        try:
            title = soup.find('h1').text
            title = title.replace('\r', '').replace('\t', '').replace('\n', '').strip()
        except:
            title = ''

        article = ''

        try:
            image = soup.find(class_ = 'a magnifier-thumb-wrapper')
            url = 'https://msk.fitnesslook.ru' + image['href']
            image_ref = f"images/{category}/{count}_fitnesslook_{article}.jpeg"
            with open(f'images/{category}/{count}_fitnesslook_{article}.jpeg', 'wb') as f:
                r = requests.get(url)
                f.write(r.content)     

        except:
            print(f'Log: no image for {article} found...')
            url = ''
            image_ref = ''

        try:
            char = soup.find(class_ = 'product-options')
            char_lst = []
            for ch in char.children:
                char_lst.append(re.sub(re.compile('\?(.*?)\:'), '', ch.text).strip())
            chars = ' '.join(char_lst)
            chars_fin = chars.split('Преимущества')[0].strip()
        except:
            chars_fin = ''



        desc = ''

        try:
            price = soup.find(class_ = 'currentProductPrice').text
            price = price.replace(' ', '')
        except:
            price =''

        try:
            cat = soup.find(class_ = 'breadcrumbs').text.split('\n\n')
            
        except:
            cat = []
        try:
            cat_1 = cat[-3].replace('\n', '').strip()
        except:
            cat_1 = ''
        try:
            cat_2 = cat[-2].replace('\n', '').strip()
        except:
            cat_2 = ''
        try:
            cat_3 = cat[-1].replace('\n', '').strip()
        except:
            cat_3 = ''

        yield  title, article, price, cat_1, cat_2, cat_3, url, desc, chars_fin, image_ref

In [4]:
df_columns = ['title', 'article','price', 'cat_1', 'cat_2', 'cat_3','url', 
              'description', 'caracteristics', 'img_ref']

df = pd.DataFrame(columns = df_columns)
save_frequency = 300
log_frequency = 10

In [5]:
import time

def to_csv(category, tot_pages):
    counter = 0  

    for item in get_item(category, tot_pages):
            data = []
            for i in range(len(df_columns)):
                data.append(item[i])
            df.loc[len(df.index)] = data
            time.sleep(0.3)

            counter += 1
            if counter % save_frequency == 0:
                df.to_csv(f'{category}_fitnesslook.csv')
                print(f'{counter} items saved in csv..')
    df.to_csv(f'{category}_fitnesslook.csv')

    print(f'\nTotal items saved: {counter}')  

In [6]:
to_csv('grebnye_trenazhery', 6)


Total items saved: 174


In [7]:
df

,title,article,price,cat_1,cat_2,cat_3,url,description,caracteristics,img_ref
0,Гребной тренажер Orlauf River в Москве,,59900,Кардиотренажеры,Гребные тренажеры,Orlauf River,https://msk.fitnesslook.ru/i/product_i/17136_1...,,Характеристики River Использование: домашнее С...,images/grebnye_trenazhery/1_fitnesslook_.jpeg
1,Гребной тренажер First Degree Fitness Apollo P...,,209900,Кардиотренажеры,Гребные тренажеры,First Degree Fitness Apollo Plus,https://msk.fitnesslook.ru/i/product_i/8689_3_...,,Характеристики Apollo Plus Система нагружения:...,images/grebnye_trenazhery/2_fitnesslook_.jpeg
2,Гребной тренажер Bronze Gym RW1100M PRO в Москве,,119990,Кардиотренажеры,Гребные тренажеры,Bronze Gym RW1100M PRO,https://msk.fitnesslook.ru/i/product_i/16213_1...,,Система нагружения: аэродинамическая Эргометр:...,images/grebnye_trenazhery/3_fitnesslook_.jpeg
3,Гребной тренажер VictoryFit VF-WR800 в Москве,,102900,Кардиотренажеры,Гребные тренажеры,VictoryFit VF-WR800,https://msk.fitnesslook.ru/i/product_i/8521_8_...,,Характеристики VF-WR800 Система нагружения: во...,images/grebnye_trenazhery/4_fitnesslook_.jpeg
4,Гребной тренажер VictoryFit VF-WR801 в Москве,,106500,Кардиотренажеры,Гребные тренажеры,VictoryFit VF-WR801,https://msk.fitnesslook.ru/i/product_i/8522_10...,,Характеристики VF-WR801 Система нагружения: во...,images/grebnye_trenazhery/5_fitnesslook_.jpeg
...,...,...,...,...,...,...,...,...,...,...
169,Гребной тренажер Fluid Rower Evolution E550 в ...,,489900,Кардиотренажеры,Гребные тренажеры,Fluid Rower Evolution E550,https://msk.fitnesslook.ru/i/product_i/17125_6...,,Характеристики Evolution E550 Система нагружен...,images/grebnye_trenazhery/170_fitnesslook_.jpeg
170,Гребной тренажер Fluid Rower Evolution E350 в ...,,419900,Кардиотренажеры,Гребные тренажеры,Fluid Rower Evolution E350,https://msk.fitnesslook.ru/i/product_i/17126_9...,,Характеристики Evolution E350 Система нагружен...,images/grebnye_trenazhery/171_fitnesslook_.jpeg
171,Гребной тренажер Fluid Rower Vortex VX-3 в Москве,,549900,Кардиотренажеры,Гребные тренажеры,Fluid Rower Vortex VX-3,https://msk.fitnesslook.ru/i/product_i/17127_6...,,Характеристики Vortex VX-3 Система нагружения:...,images/grebnye_trenazhery/172_fitnesslook_.jpeg
172,Гребной тренажер Fluid Rower Vortex VX-2 в Москве,,499900,Кардиотренажеры,Гребные тренажеры,Fluid Rower Vortex VX-2,https://msk.fitnesslook.ru/i/product_i/17128_9...,,Характеристики Vortex VX-2 Система нагружения:...,images/grebnye_trenazhery/173_fitnesslook_.jpeg
